#Sources
- Chat Bubbles CSS
    - https://chatableapps.com/technology/coding-css-chat-bubbles-a-step-by-step-guide-to-creating-stylish-chat-interfaces/
- generators
    - https://docs.python.org/3/tutorial/classes.html
- gradio
    - https://www.machinelearningnuggets.com/gradio-tutorial/
- html
    - https://borstch.com/snippet/negotiation-simulation-game

In [9]:
import gradio as gr
import json
import time
import os

#load the file and get the simulations
def load_conversations(file_path):
    """Load JSONL-formatted conversations from file_path."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    conversations = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            conversations.append(json.loads(line))
    return conversations

#JSONL file
negotiation_chat_file = "negotiation_chats_selfplay (1).jsonl"
conversations = load_conversations(negotiation_chat_file)

conversation_id = "4e229c40-bcae-4efc-8c3f-2032034210a5"  #Enter conversation ID we want to show 
chat_delay = 4.0  #Adjustable delay




#retrieve the negotiation by the ID
def get_conversation_by_id(convo_id):
    """Return the list of chats for a given conversation ID."""
    for convo in conversations:
        if convo.get('ConversationID') == convo_id:
            return convo.get('Negotiation', [])
    return []


#runs the selected negotiation/simulation of self play
def simulate_conversation():
    """
    Simulate the conversation for the fixed conversation_id, 
    yielding updates after each message.
    """
    messages = get_conversation_by_id(conversation_id)
    if not messages:
        yield "<div>No conversation found with the given Conversation ID.</div>"
        return

    chat_html = ""
    for msg in messages:
        #Create message bubbles
        if msg['role'] == "Employee":
            chat_html += f"""
            <div style="text-align: left; margin: 10px;">
                <div style="display: inline-block; padding: 10px; border-radius: 10px; 
                            background-color: #d1f4ff; max-width: 60%; font-size: 14px;">
                    <b>{msg['role']}</b>: {msg['content']}
                </div>
            </div>
            """
        elif msg['role'] == "HR Manager":
            chat_html += f"""
            <div style="text-align: right; margin: 10px;">
                <div style="display: inline-block; padding: 10px; border-radius: 10px; 
                            background-color: #e6ffe6; max-width: 60%; font-size: 14px;">
                    <b>{msg['role']}</b>: {msg['content']}
                </div>
            </div>
            """

        #Yield the updated chat state to the UI
        yield gr.update(value=chat_html)
        time.sleep(chat_delay)

    #After all messages have been displayed, show simulation has ened 
    chat_html += """
    <div style="text-align: center; margin: 20px; font-size: 14px;">
        Negotiation simulation ended.
    </div>
    """
    yield gr.update(value=chat_html)

#Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Self-Play Negotiation Simulation Demo")
    gr.Markdown("Click 'Start Simulation' to begin the simulation.")
    
    start_button = gr.Button("Start Simulation")
    output_box = gr.HTML(label="Live Chat Simulation")

    #Trigger live conversation simulation
    start_button.click(simulate_conversation, inputs=None, outputs=output_box)

#Launch the Gradio interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
